In [2]:
# vamos a importar requests para hacer las peticiones http basicamente estamos bajando las paginas web
import requests 

# importamos Bs para analizar el html y extraer informacion de el mismo parsear
from bs4 import BeautifulSoup


# importamos pandas para guardar los resultados en una tabla osea un dataframe
import pandas as pd 

# esta es la url base del sitio que vamos a scrapear
BASE_URL = "http://books.toscrape.com/"

In [3]:
# estamos haciendo peticion a la pp
response = requests.get(BASE_URL)

#creamos un objeto para parsear el html de la pagina
soup = BeautifulSoup(response.text, "html.parser")

#buscamos el bloque HTML donde estan todas las categorias
categorias_html = soup.find("ul", class_="nav-list").find("ul").find_all("a")

#vamos a crear un dicciconario {nombre_categoria: link_categoria}
categorias = {}
for c in categorias_html:
    nombre = c.text.strip()  # nombre de la categoria ej: Travel, Mistery, etc 
    link = BASE_URL + c["href"] # es el enlace absoluto a la categoria 
    categorias[nombre] = link

# Mostramos cuántas categorías encontramos (deben ser 50 incluyendo 'Books', o 49 sin la principal)
print("Categorías encontradas:", len(categorias))
print(list(categorias.keys())[:5])   # Mostramos las primeras 5 categorías como ejemplo

Categorías encontradas: 50
['Travel', 'Mystery', 'Historical Fiction', 'Sequential Art', 'Classics']


In [ ]:
#listas vacias para guardar la informacion de cada libro #CORAZON DEL SCRAPING

titulos = [] #cajones donde vamos guardando cada atributo 
precios = []
disponibilidades = []
ratings = []
categorias_lista = []

for nombre_cat, url_cat in categorias.items():
    print(f"Scrapeando categoría: {nombre_cat}")  # Feedback en consola

    #empezamos desde la primera pagina de esa categoria
    next_page = url_cat

    #bucle para recorrer todas las paginas dentro de esa categoria
    while next_page:

        #hacemos la peticion http a la pagina
        r = requests.get(next_page)
        s = BeautifulSoup(r.text, "html.parser")

        #buscamos todos los libros dentro de la pagina 
        libros = s.find_all("article", class_="product_pod") #findall todas las coincidencias una lista

         # Recorremos cada libro encontrado
        for libro in libros:
            titulo = libro.h3.a["title"]   # Título del libro
            precio = libro.find("p", class_="price_color").text  # Precio (ej: '£53.74')
            disponibilidad = libro.find("p", class_="instock availability").text.strip()  # Disponibilidad limpia espacios y salto de linea
            rating = libro.find("p", class_="star-rating")["class"][1]  # Rating (ej: 'Three', 'Five')

            # Guardamos la información en nuestras listas
            titulos.append(titulo)
            precios.append(precio)
            disponibilidades.append(disponibilidad)
            ratings.append(rating)
            categorias_lista.append(nombre_cat)
        
        # Verificamos si hay un botón "next" para ir a la siguiente página
        next_btn = s.find("li", class_="next")
        if next_btn:
            # Armamos el link de la siguiente página
            next_page = url_cat.replace("index.html", "") + next_btn.a["href"]
        else:
            next_page = None  # Si no hay más páginas, salimos del bucle


Scrapeando categoría: Travel
Scrapeando categoría: Mystery
Scrapeando categoría: Historical Fiction
Scrapeando categoría: Sequential Art
Scrapeando categoría: Classics
Scrapeando categoría: Philosophy
Scrapeando categoría: Romance
Scrapeando categoría: Womens Fiction
Scrapeando categoría: Fiction
Scrapeando categoría: Childrens
Scrapeando categoría: Religion
Scrapeando categoría: Nonfiction


In [4]:
#creamos un dataframe con todos los datos recolectados
df = pd.DataFrame({
    "titulo": titulos,
    "precios": precios,
    "disponibilidad": disponibilidades,
    "rating": ratings,
    "categoria": categorias_lista
    })

# Mostramos cuántos libros se obtuvieron en total
print("Libros totales encontrados:", len(df))

# Vemos los primeros 5 registros para comprobar
df.head()
print(df.columns.tolist())
display(df.head())

Libros totales encontrados: 1000
['titulo', 'precios', 'disponibilidad', 'rating', 'categoria']


,titulo,precios,disponibilidad,rating,categoria
0,It's Only the Himalayas,Â£45.17,In stock,Two,Travel
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,Â£49.43,In stock,Four,Travel
2,See America: A Celebration of Our National Par...,Â£48.87,In stock,Three,Travel
3,Vagabonding: An Uncommon Guide to the Art of L...,Â£36.94,In stock,Two,Travel
4,Under the Tuscan Sun,Â£37.33,In stock,Three,Travel


In [ ]:
# Renombrar 'precios' -> 'precio' 
if "precios" in df.columns and "precio" not in df.columns:
    df = df.rename(columns={"precios": "precio"})

# ✅ Limpiar símbolo de libra y convertir a float
df["precio"] = (
    df["precio"].astype(str)
        .str.replace("Â£", "", regex=False) #como texto literal no como expresion regular
        .str.replace("£", "", regex=False)
        .astype(float) #convierte de texto numerico str a decimal
)

print("Columnas después de limpiar/renombrar:", df.columns.tolist()) #formato python listas
display(df.head())


Columnas después de limpiar/renombrar: ['titulo', 'precio', 'disponibilidad', 'rating', 'categoria']


,titulo,precio,disponibilidad,rating,categoria
0,It's Only the Himalayas,45.17,In stock,Two,Travel
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,In stock,Four,Travel
2,See America: A Celebration of Our National Par...,48.87,In stock,Three,Travel
3,Vagabonding: An Uncommon Guide to the Art of L...,36.94,In stock,Two,Travel
4,Under the Tuscan Sun,37.33,In stock,Three,Travel


In [ ]:

#Procedemos a crear la base de datos y tablas DDL
import sqlite3

#conexion a la base de datos se crea un archivo llamado books.db en la carpeta
conexion = sqlite3.connect("books.db")
cursor = conexion.cursor() #instrucciones sql dentro de la base

#creamos la tabla de categorias
cursor.execute("""
CREATE TABLE IF NOT EXISTS categorias (
    id_categoria INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre_categoria TEXT UNIQUE
)

""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS libros (
    id_libro INTEGER PRIMARY KEY AUTOINCREMENT,
    titulo TEXT,
    precio REAL,
    disponibilidad TEXT,
    rating TEXT,
    id_categoria INTEGER,
    FOREIGN KEY (id_categoria) REFERENCES categorias(id_categoria)
)

""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS autores (
    id_autor INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre_autor TEXT UNIQUE
)

""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS libro_autor (
    id_libro INTEGER,
    id_autor INTEGER,
    PRIMARY KEY (id_libro, id_autor),
    FOREIGN KEY (id_libro) REFERENCES libros(id_libro),
    FOREIGN KEY (id_autor) REFERENCES autores(id_autor)
)
""")

print(" Tablas creadas correctamente")

 Tablas creadas correctamente


In [ ]:
import sqlite3

conexion = sqlite3.connect("books.db", timeout=10)
cursor = conexion.cursor()
print("Conexión y cursor abiertos de nuevo") #solo confirma la conexion


Conexión y cursor abiertos de nuevo


In [ ]:
cursor.execute("SELECT COUNT(*) FROM categorias")
print("Total categorías en la base:", cursor.fetchone()[0])


Total categorías en la base: 50


In [ ]:
categorias_unicas = df["categoria"].unique() #valores unicos sin repetidos

for nombre_categoria in categorias_unicas:
    cursor.execute(
        "INSERT OR IGNORE INTO categorias (nombre_categoria) VALUES (?)",
        (nombre_categoria,)
    )
#hasta no hacer commit los insert quedan en memoria y los datos insertados son guardados en el archivo books.db 
conexion.commit()
print("Categorías insertadas:", len(categorias_unicas))



Categorías insertadas: 50


In [ ]:
for _, fila in df.iterrows(): #indice _
    # Obtener el id_categoria de la tabla categorias
    cursor.execute("SELECT id_categoria FROM categorias WHERE nombre_categoria = ?", (fila["categoria"],))
    id_categoria = cursor.fetchone()[0]

    # Insertar libro en la tabla libros
    cursor.execute("""
        INSERT INTO libros (titulo, precio, disponibilidad, rating, id_categoria)
        VALUES (?, ?, ?, ?, ?)
    """, (fila["titulo"], fila["precio"], fila["disponibilidad"], fila["rating"], id_categoria))


In [11]:
# Creamos 20 autores ficticios
for i in range(1, 21):
    nombre_autor = f"Autor {i}"
    cursor.execute("INSERT OR IGNORE INTO autores (nombre_autor) VALUES (?)", (nombre_autor,))


In [ ]:
##consulta de prueba
cursor.execute("""
SELECT libros.titulo, categorias.nombre_categoria
FROM libros
JOIN categorias ON libros.id_categoria = categorias.id_categoria
LIMIT 5
""")
print("Ejemplo de libros con su categoría:")
print(cursor.fetchall()) 


Ejemplo de libros con su categoría:
[("It's Only the Himalayas", 'Travel'), ('Full Moon over Noahâ\x80\x99s Ark: An Odyssey to Mount Ararat and Beyond', 'Travel'), ('See America: A Celebration of Our National Parks & Treasured Sites', 'Travel'), ('Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel', 'Travel'), ('Under the Tuscan Sun', 'Travel')]


In [ ]:
import random

# Obtenemos todos los id_libro
cursor.execute("SELECT id_libro FROM libros")
lista_libros = [fila[0] for fila in cursor.fetchall()]

# Obtenemos todos los id_autor
cursor.execute("SELECT id_autor FROM autores")
lista_autores = [fila[0] for fila in cursor.fetchall()]

# Relacionamos cada libro con entre 1 y 3 autores al azar
for id_libro in lista_libros:
    autores_asignados = random.sample(lista_autores, random.randint(1, 3)) #AL AZAR AUTORES PREVIAMENTE DEFINIDOS
    for id_autor in autores_asignados:
        cursor.execute("INSERT OR IGNORE INTO libro_autor (id_libro, id_autor) VALUES (?, ?)", (id_libro, id_autor))

# Guardamos los cambios en la base de datos
conexion.commit()
print("Datos insertados correctamente")


Datos insertados correctamente


In [14]:
#Probamos que funciona
# Ver algunos libros con su categoría
cursor.execute("""
SELECT libros.titulo, categorias.nombre_categoria
FROM libros
JOIN categorias ON libros.id_categoria = categorias.id_categoria
LIMIT 5
""")
print(" Libros con categoría:")
print(cursor.fetchall())

# Ver algunos libros con sus autores
cursor.execute("""
SELECT libros.titulo, autores.nombre_autor
FROM libros
JOIN libro_autor ON libros.id_libro = libro_autor.id_libro
JOIN autores ON libro_autor.id_autor = autores.id_autor
LIMIT 10
""")
print("\nLibros con autores:")
print(cursor.fetchall())


 Libros con categoría:
[("It's Only the Himalayas", 'Travel'), ('Full Moon over Noahâ\x80\x99s Ark: An Odyssey to Mount Ararat and Beyond', 'Travel'), ('See America: A Celebration of Our National Parks & Treasured Sites', 'Travel'), ('Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel', 'Travel'), ('Under the Tuscan Sun', 'Travel')]

Libros con autores:
[("It's Only the Himalayas", 'Autor 18'), ("It's Only the Himalayas", 'Autor 9'), ('Full Moon over Noahâ\x80\x99s Ark: An Odyssey to Mount Ararat and Beyond', 'Autor 7'), ('Full Moon over Noahâ\x80\x99s Ark: An Odyssey to Mount Ararat and Beyond', 'Autor 18'), ('See America: A Celebration of Our National Parks & Treasured Sites', 'Autor 5'), ('See America: A Celebration of Our National Parks & Treasured Sites', 'Autor 12'), ('Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel', 'Autor 3'), ('Under the Tuscan Sun', 'Autor 14'), ('A Summer In Europe', 'Autor 1'), ('The Great Railway Bazaar', 'Autor 20')]


#DIAGRAMA UML
```mermaid
erDiagram
    CATEGORIAS {
        int id_categoria PK
        string nombre_categoria
    }

    LIBROS {
        int id_libro PK
        string titulo
        float precio
        string disponibilidad
        string rating
        int id_categoria FK
    }

    AUTORES {
        int id_autor PK
        string nombre_autor
    }

    LIBRO_AUTOR {
        int id_libro FK
        int id_autor FK
    }

    CATEGORIAS ||--o{ LIBROS : contiene
    LIBROS ||--o{ LIBRO_AUTOR : tiene
    AUTORES ||--o{ LIBRO_AUTOR : escribe
```


CATEGORIAS (id_categoria, nombre_categoria)
    |
    |---< LIBROS (id_libro, titulo, precio, disponibilidad, rating, id_categoria)
              |
              |---< LIBRO_AUTOR (id_libro, id_autor) >--- AUTORES (id_autor, nombre_autor)


In [ ]:
# Empezamos con las consultas faciles
#Queremos ver todos los libros

import pandas as pd 

df = pd.read_sql_query("SELECT * FROM libros;", conexion) #DJ QUE PIDE LA CANCION CONSULTA SQL Y LA CONVIERTE EN MUSICA (DATAFRAME)
display(df.head())

#queremos ver solo titulo y precio
df = pd.read_sql_query("SELECT titulo, precio FROM libros;", conexion)
display(df.head())


,id_libro,titulo,precio,disponibilidad,rating,id_categoria
0,1,It's Only the Himalayas,45.17,In stock,Two,1
1,2,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,In stock,Four,1
2,3,See America: A Celebration of Our National Par...,48.87,In stock,Three,1
3,4,Vagabonding: An Uncommon Guide to the Art of L...,36.94,In stock,Two,1
4,5,Under the Tuscan Sun,37.33,In stock,Three,1


,titulo,precio
0,It's Only the Himalayas,45.17
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43
2,See America: A Celebration of Our National Par...,48.87
3,Vagabonding: An Uncommon Guide to the Art of L...,36.94
4,Under the Tuscan Sun,37.33


In [16]:
#Ahora vemos los librios con precios menor a 20 por ejemplo.

pd.read_sql_query("SELECT titulo, precio FROM libros WHERE precio < 20;", conexion)


,titulo,precio
0,"In a Dark, Dark Wood",19.63
1,A Murder in Time,16.64
2,That Darkness (Gardiner and Renner #1),13.92
3,Tastes Like Fear (DI Marnie Rome #3),10.69
4,A Study in Scarlet (Sherlock Holmes #1),16.73
...,...,...
779,Logan Kade (Fallen Crest High #5.5),13.12
780,You Are a Badass: How to Stop Doubting Your Gr...,12.08
781,The Girl You Left Behind (The Girl You Left Be...,15.79
782,Dark Notes,19.19


In [17]:
#libros con rating FOUR or FIVE
pd.read_sql_query("SELECT titulo, rating FROM libros WHERE rating IN ('Four','Five');", conexion)



,titulo,rating
0,Full Moon over Noahâs Ark: An Odyssey to Mou...,Four
1,A Year in Provence (Provence #1),Four
2,"1,000 Places to See Before You Die",Five
3,Sharp Objects,Four
4,The Past Never Ends,Four
...,...,...
1495,Suzie Snowflake: One beautiful flake (a self-e...,Five
1496,10-Day Green Smoothie Cleanse: Lose Up to 15 P...,Five
1497,The Art and Science of Low Carbohydrate Living,Five
1498,Why the Right Went Wrong: Conservatism--From G...,Four


In [ ]:
# Libros que en el título tengan la palabra 'Travel' y filtramos por palabra si queremos que trate de algun tema en especifico
pd.read_sql_query("SELECT titulo, precio FROM libros WHERE titulo LIKE '%Travel%';", conexion)

,titulo,precio
0,Vagabonding: An Uncommon Guide to the Art of L...,36.94
1,Neither Here nor There: Travels in Europe,38.95
2,Travels with Charley: In Search of America,57.82
3,Ship Leaves Harbor: Essays on Travel by a Reco...,30.60
4,The Travelers,15.77
5,Vagabonding: An Uncommon Guide to the Art of L...,36.94
6,Neither Here nor There: Travels in Europe,38.95
7,Travels with Charley: In Search of America,57.82
8,Ship Leaves Harbor: Essays on Travel by a Reco...,30.60
9,The Travelers,15.77


In [ ]:
# Libros con su categoría (JOIN directo) #alias l. c 
pd.read_sql_query("""
SELECT l.id_libro, l.titulo, c.nombre_categoria 
FROM LIBROS l
JOIN CATEGORIAS c ON l.id_categoria = c.id_categoria
ORDER BY c.nombre_categoria, l.titulo;
""", conexion)


,id_libro,titulo,nombre_categoria
0,977,Logan Kade (Fallen Crest High #5.5),Academic
1,1977,Logan Kade (Fallen Crest High #5.5),Academic
2,2977,Logan Kade (Fallen Crest High #5.5),Academic
3,3977,Logan Kade (Fallen Crest High #5.5),Academic
4,683,#GIRLBOSS,Add a comment
...,...,...,...
3995,3755,Wild Swans,Young Adult
3996,774,"Will Grayson, Will Grayson (Will Grayson, Will...",Young Adult
3997,1774,"Will Grayson, Will Grayson (Will Grayson, Will...",Young Adult
3998,2774,"Will Grayson, Will Grayson (Will Grayson, Will...",Young Adult


In [20]:
# Libros con sus autores (Many-to-Many a través de LIBRO_AUTOR) JOIN SOLO
pd.read_sql_query("""
SELECT l.titulo, a.nombre_autor
FROM LIBROS l
JOIN LIBRO_AUTOR la ON l.id_libro = la.id_libro
JOIN AUTORES a ON la.id_autor = a.id_autor
ORDER BY l.titulo, a.nombre_autor;
""", conexion)


,titulo,nombre_autor
0,"""Most Blessed of the Patriarchs"": Thomas Jeffe...",Autor 1
1,"""Most Blessed of the Patriarchs"": Thomas Jeffe...",Autor 10
2,"""Most Blessed of the Patriarchs"": Thomas Jeffe...",Autor 10
3,"""Most Blessed of the Patriarchs"": Thomas Jeffe...",Autor 11
4,"""Most Blessed of the Patriarchs"": Thomas Jeffe...",Autor 11
...,...,...
18071,salt.,Autor 7
18072,salt.,Autor 7
18073,salt.,Autor 7
18074,salt.,Autor 8


In [21]:
#  Libros, autores y categoría juntos + filtro por precio join solo
pd.read_sql_query("""
SELECT l.titulo, a.nombre_autor, c.nombre_categoria, l.precio
FROM LIBROS l
JOIN LIBRO_AUTOR la ON l.id_libro = la.id_libro
JOIN AUTORES a ON la.id_autor = a.id_autor
JOIN CATEGORIAS c ON l.id_categoria = c.id_categoria
WHERE l.precio < 20
ORDER BY l.precio ASC;
""", conexion)


,titulo,nombre_autor,nombre_categoria,precio
0,An Abundance of Katherines,Autor 2,Young Adult,10.00
1,An Abundance of Katherines,Autor 15,Young Adult,10.00
2,An Abundance of Katherines,Autor 1,Young Adult,10.00
3,An Abundance of Katherines,Autor 17,Young Adult,10.00
4,An Abundance of Katherines,Autor 10,Young Adult,10.00
...,...,...,...,...
3556,Lumberjanes Vol. 3: A Terrible Plan (Lumberjan...,Autor 4,Sequential Art,19.92
3557,Lumberjanes Vol. 3: A Terrible Plan (Lumberjan...,Autor 15,Sequential Art,19.92
3558,Lumberjanes Vol. 3: A Terrible Plan (Lumberjan...,Autor 12,Sequential Art,19.92
3559,Lumberjanes Vol. 3: A Terrible Plan (Lumberjan...,Autor 20,Sequential Art,19.92


In [22]:
# Precio promedio por categoría join + group by
pd.read_sql_query("""
SELECT c.nombre_categoria, ROUND(AVG(l.precio), 2) AS precio_promedio
FROM LIBROS l
JOIN CATEGORIAS c ON l.id_categoria = c.id_categoria
GROUP BY c.nombre_categoria
ORDER BY precio_promedio DESC;
""", conexion)

,nombre_categoria,precio_promedio
0,Suspense,58.33
1,Novels,54.81
2,Politics,53.61
3,Health,51.45
4,New Adult,46.38
5,Christian,42.50
6,Sports and Games,41.17
7,Self Help,40.62
8,Travel,39.79
9,Fantasy,39.59


In [ ]:
# Precio mínimo y máximo por categoría JOIN + GROUP BY para tener un aproximado de cuanto gastariamos por categoria
pd.read_sql_query("""
SELECT c.nombre_categoria,
       MIN(l.precio) AS precio_min,
       MAX(l.precio) AS precio_max
FROM LIBROS l
JOIN CATEGORIAS c ON l.id_categoria = c.id_categoria
GROUP BY c.nombre_categoria
ORDER BY c.nombre_categoria;
""", conexion)


,nombre_categoria,precio_min,precio_max
0,Academic,13.12,13.12
1,Add a comment,10.02,59.15
2,Adult Fiction,15.36,15.36
3,Art,10.29,49.05
4,Autobiography,10.93,59.04
5,Biography,16.85,48.19
6,Business,12.61,51.74
7,Childrens,10.62,58.08
8,Christian,25.77,54.00
9,Christian Fiction,17.97,49.97


In [ ]:
# Conteo de libros por rating interesante saber cuantos libros van del 1 al ..
pd.read_sql_query("""
SELECT l.rating, COUNT(*) AS total
FROM LIBROS l
GROUP BY l.rating
ORDER BY total DESC;
""", conexion)


,rating,total
0,One,904
1,Three,812
2,Two,784
3,Five,784
4,Four,716


In [26]:
#FUNCION DE VENTANA
import pandas as pd

consulta = """
SELECT 
    c.nombre_categoria,
    l.titulo,
    l.precio,
    RANK() OVER (PARTITION BY c.id_categoria ORDER BY l.precio DESC) AS ranking_precio
FROM LIBROS l
JOIN CATEGORIAS c ON l.id_categoria = c.id_categoria
ORDER BY c.nombre_categoria, ranking_precio;
"""

pd.read_sql_query(consulta, conexion)



,nombre_categoria,titulo,precio,ranking_precio
0,Academic,Logan Kade (Fallen Crest High #5.5),13.12,1
1,Academic,Logan Kade (Fallen Crest High #5.5),13.12,1
2,Academic,Logan Kade (Fallen Crest High #5.5),13.12,1
3,Academic,Logan Kade (Fallen Crest High #5.5),13.12,1
4,Add a comment,The Gray Rhino: How to Recognize and Act on th...,59.15,1
...,...,...,...,...
3995,Young Adult,The Darkest Corners,11.33,209
3996,Young Adult,An Abundance of Katherines,10.00,213
3997,Young Adult,An Abundance of Katherines,10.00,213
3998,Young Adult,An Abundance of Katherines,10.00,213


In [ ]:
#SUBCONSULTA (libros mas caros del promedio de su categoria) para saber que comprar

import pandas as pd

consulta2 = """
SELECT 
    l.titulo,
    l.precio,
    c.nombre_categoria
FROM LIBROS l
JOIN CATEGORIAS c ON l.id_categoria = c.id_categoria
WHERE l.precio > (
    SELECT AVG(l2.precio)
    FROM LIBROS l2
    WHERE l2.id_categoria = l.id_categoria
);
"""

pd.read_sql_query(consulta2, conexion)


,titulo,precio,nombre_categoria
0,It's Only the Himalayas,45.17,Travel
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,Travel
2,See America: A Celebration of Our National Par...,48.87,Travel
3,A Summer In Europe,44.34,Travel
4,A Year in Provence (Provence #1),56.88,Travel
...,...,...,...
2023,(Un)Qualified: How God Uses Broken People to D...,54.00,Christian
2024,Crazy Love: Overwhelmed by a Relentless God,47.72,Christian
2025,"Eat Fat, Get Thin",54.07,Health
2026,The Art and Science of Low Carbohydrate Living,52.98,Health


In [28]:
#HAVING (magia oscura)
import pandas as pd

consulta3 = """
SELECT 
    a.nombre_autor,
    COUNT(*) AS cantidad_libros
FROM AUTORES a
JOIN LIBRO_AUTOR la ON a.id_autor = la.id_autor
JOIN LIBROS l ON la.id_libro = l.id_libro
GROUP BY a.nombre_autor
HAVING COUNT(*) > 1
ORDER BY cantidad_libros DESC;
"""

pd.read_sql_query(consulta3, conexion)


,nombre_autor,cantidad_libros
0,Autor 2,949
1,Autor 18,939
2,Autor 7,938
3,Autor 6,931
4,Autor 3,927
5,Autor 9,923
6,Autor 19,913
7,Autor 15,911
8,Autor 8,910
9,Autor 1,910


In [30]:
#full scan dolorosamente lenta
consulta_lenta = """
SELECT *
FROM LIBROS
WHERE id_categoria = 3;
"""

pd.read_sql_query(consulta_lenta, conexion)




,id_libro,titulo,precio,disponibilidad,rating,id_categoria
0,44,Tipping the Velvet,53.74,In stock,One,3
1,45,Forever and Forever: The Courtship of Henry Lo...,29.69,In stock,Three,3
2,46,A Flight of Arrows (The Pathfinders #2),55.53,In stock,Five,3
3,47,The House by the Lake,36.95,In stock,One,3
4,48,Mrs. Houdini,30.25,In stock,Five,3
...,...,...,...,...,...,...
99,3065,While You Were Mine,41.32,In stock,Five,3
100,3066,The Secret Healer,34.56,In stock,Three,3
101,3067,Starlark,25.83,In stock,Three,3
102,3068,Lost Among the Living,27.70,In stock,Four,3


In [ ]:
crear_indice = "CREATE INDEX IF NOT EXISTS idx_libros_categoria ON LIBROS(id_categoria);"
conexion.execute(crear_indice)
#%%time
consulta_rapida = """
SELECT *
FROM LIBROS
WHERE id_categoria = 3;
"""

pd.read_sql_query(consulta_rapida, conexion)

#probar con time


,id_libro,titulo,precio,disponibilidad,rating,id_categoria
0,44,Tipping the Velvet,53.74,In stock,One,3
1,45,Forever and Forever: The Courtship of Henry Lo...,29.69,In stock,Three,3
2,46,A Flight of Arrows (The Pathfinders #2),55.53,In stock,Five,3
3,47,The House by the Lake,36.95,In stock,One,3
4,48,Mrs. Houdini,30.25,In stock,Five,3
...,...,...,...,...,...,...
99,3065,While You Were Mine,41.32,In stock,Five,3
100,3066,The Secret Healer,34.56,In stock,Three,3
101,3067,Starlark,25.83,In stock,Three,3
102,3068,Lost Among the Living,27.70,In stock,Four,3
